В данной задаче нам необходимо реализовать маленькую, не сильно требовательную рекомендательную систему. В качестве решения, по моему, можно использовать ту же библиотеку surprise, или код, написанный для задачи 243. РекомендательнаяСистемаSVD, или любую другую библиотеку для рекомендательных систем по типу implicit, lightFM. Большинство этих решений пройдет, проверено)
В итоге, нам на основе существующих оценок необходимо составить матрицу user-item (в которой содержатся id-пользователя, id-заведения и оценки на пересечениях строк и столбцов), а затем с помощью любой библиотеки провести факторизацию и рекомендовать каждому пользователю от 1 до 100 заведений из города, в котором он НЕ живет.
Я в коде ниже выбрал библиотеку implicit и каждому пользователю рекомендовал 40 заведений.

P.S. Когда я пытался сделать эту задачу у меня не проходило огромное количество решений из-за того, что был сломан чекер, поэтому я углублялся все дальше, а тетрадка засорялась все сильнее и плодились ее копии с разными библиотеками. Каково же было мое удивление, когда я зашёл отправить очередное решение и увидел, что задача зачтена, а вместе с тем 80% отправленных решений также зачтены. В итоге, в качестве примера я выбрал не самое сложное, но и не самое легкое в понимании решение, для которого у меня сохранилась тетрадка в +- адекватном виде.
P.P.S. Задача настолько не требовательное, что ходят слухи, что можно рекомендовать просто самые популярные заведения в городе или около того, что в целом и логично, но )))

In [1]:
import  pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
reviews = pd.read_csv('reviews.csv', index_col=0)
reviews['rating'] = np.array(reviews.rating.fillna(0) + 1)
orgs = pd.read_csv('organisations.csv')
users = pd.read_csv('users.csv')

# encode users ids as numeric
reviews = reviews.merge(users, on='user_id')
reviews = reviews.rename({'city': 'user_city'}, axis=1)

# # encode orgs ids as numeric
reviews = reviews.merge(orgs[['org_id', 'city','rubrics_id','features_id','rating','average_bill']], on='org_id')
reviews = reviews.rename({'city': 'org_city'}, axis=1)

In [4]:
import scipy

rows, r_pos = np.unique(reviews.values[:,1], return_inverse=True)
cols, c_pos = np.unique(reviews.values[:,0], return_inverse=True)

In [5]:
interactions_sparse = scipy.sparse.csr_matrix((np.array(reviews['rating_x'],dtype=np.float64),(r_pos, c_pos)))

In [7]:
from implicit.evaluation import train_test_split

train, test_set = train_test_split(interactions_sparse,0.99)

In [8]:
from implicit.als import AlternatingLeastSquares

machine = AlternatingLeastSquares(factors = 512, iterations=10)

machine.fit(train)

  0%|          | 0/10 [00:00<?, ?it/s]

In [9]:
reviews['r_pos'] = r_pos
reviews['c_pos'] = c_pos

In [10]:
test = pd.read_csv('test_users.csv',index_col=0)

In [11]:
test = test.merge(reviews,how='left')[['user_id','user_city','r_pos']].drop_duplicates('user_id')

In [12]:
orgs = reviews[['org_id','org_city','c_pos']].drop_duplicates('org_id')

In [13]:
res = []
for indx,row in test.iterrows():
    user_city = row['user_city']
    user_pos = row['r_pos']
    to_exclude = orgs[orgs.org_city != user_city].c_pos.tolist()
    try:
        recommend_index, scores = machine.recommend(user_pos,
                                                    interactions_sparse[user_pos],
                                                    items=to_exclude,
                                                    filter_already_liked_items=True,
                                                    N=40)
        res.append((row['user_id'],
                    list(cols[recommend_index])))
    except:
        res.append((row['user_id'],[]))


In [14]:
answers = pd.DataFrame(res,columns=['user_id','target'])

In [15]:
answers.to_csv('answers.csv')